# FASTA

This notebook briefly explores the FASTA format, a very common format for storing DNA sequences. FASTA is the preferred format for storing reference genomes.

FASTA and FASTQ are rather similar, but FASTQ is almost always used for storing sequencing reads (with associated quality values), whereas FASTA is used for storing all kinds of DNA, RNA or protein sequencines (without associated quality values).


**Basic format**

Here is the basic format:

    >sequence1_short_name with optional additional info after whitespace
    ACATCACCCCATAAACAAATAGGTTTGGTCCTAGCCTTTCTATTAGCTCTTAGTAAGATTACACATGCAA
    GCATCCCCGTTCCAGTGAGTTCACCCTCTAAATCACCACGATCAAAAGGAACAAGCATCAAGCACGCAGC
    AATGCAGCTCAAAACGCTTAGCCTAGCCACACCCCCACGGGAAACAGCAGTGAT
    >sequence2_short_name with optional additional info after whitespace
    GCCCCAAACCCACTCCACCTTACTACCAGACAACCTTAGCCAAACCATTTACCCAAATAAAGTATAGGCG
    ATAGAAATTGAAACCTGGCGCAATAGATATAGTACCGCAAGGGAAAGATGAAAAATTATAACCAAGCATA
    ATATAG

A line starting with a > (greater-than) sign indicates the beginning of a new sequence and specifies its name. Take the first line above. Everything after the > up to and excluding the first whitespace character (sequence1_short_name), is the "short name." Everything after the > up to the end of the line (sequence1_short_name with optional additional info after whitespace) is the "long name." We usually use the short name when referring to FASTA sequences.

The next three lines consists of several nucleotides. There is a maximum number of nucleotides permitted per line; in this case, it is 70. If the sequence is longer then 70 nucleotides, it "wraps" down to the next line. Not every FASTA file uses the same maximum, but a given FASTA file must use the same maximum throughout the file.

## Parsing FASTA 

Here is a simple function for parsing a FASTA file into a Python dictionary. The dictionary maps short names to corresponding nucleotide strings (with whitespace removed).

In [ ]:
def parse_fasta(fh):
    fa = {}
    current_short_name = None
    # Part 1: compile list of lines per sequence
    for ln in fh:
        if ln[0] == '>':
            # new name line; remember current sequence's short name
            long_name = ln[1:].rstrip()
            current_short_name = long_name.split()[0]
            fa[current_short_name] = []
        else:
            # append nucleotides to current sequence
            fa[current_short_name].append(ln.rstrip())
    # Part 2: join lists into strings
    for short_name, nuc_list in fa.items():
        # join this sequence's lines into one long string
        fa[short_name] = ''.join(nuc_list)
    return fa

Testing the function by running it on the simple FASTA file:

In [ ]:
from io import StringIO

fasta_example = StringIO(
'''>sequence1_short_name with optional additional info after whitespace
ACATCACCCCATAAACAAATAGGTTTGGTCCTAGCCTTTCTATTAGCTCTTAGTAAGATTACACATGCAA
GCATCCCCGTTCCAGTGAGTTCACCCTCTAAATCACCACGATCAAAAGGAACAAGCATCAAGCACGCAGC
AATGCAGCTCAAAACGCTTAGCCTAGCCACACCCCCACGGGAAACAGCAGTGAT
>sequence2_short_name with optional additional info after whitespace
GCCCCAAACCCACTCCACCTTACTACCAGACAACCTTAGCCAAACCATTTACCCAAATAAAGTATAGGCG
ATAGAAATTGAAACCTGGCGCAATAGATATAGTACCGCAAGGGAAAGATGAAAAATTATAACCAAGCATA
ATATAG''')
parsed_fa = parse_fasta(fasta_example)
parsed_fa

## Indexed FASTA

Say you have one or more big FASTA files (e.g. the entire human reference genome) and you'd like to access those files "randomly," peeking at substrings here and there without any regular access pattern. Maybe you're mimicking a sequencing machine, reading snippets of DNA here and there.

You could start by using the parse_fasta function defined above to parse the FASTA files. Then, to access a substring, do as follows:

In [ ]:
parsed_fa['sequence2_short_name'][100:130]

Accessing a substring in this way is very fast and simple. The downside is that you've stored all of the sequences in memory. If the FASTA files are really big, this takes lots of valuable memory. This may or may not be a good trade.

An alternative is to load only the portions of the FASTA files that you need, when you need them. For this to be practical, we have to have a way of "jumping" to the specific part of the specific FASTA file that you're intersted in.

Fortunately, there is a standard way of indexing a FASTA file, popularized by the faidx tool in SAMtools. When you have such an index, it's easy to calculate exactly where to jump to when you want to extract a specific substring. Here is some Python to create such an index:

In [ ]:
def index_fasta(fh):
    index = []
    current_short_name = None
    current_byte_offset, running_seq_length, running_byte_offset = 0, 0, 0
    line_length_including_ws, line_length_excluding_ws = 0, 0
    for ln in fh:
        ln_stripped = ln.rstrip()
        running_byte_offset += len(ln)
        if ln[0] == '>':
            if current_short_name is not None:
                index.append((current_short_name, running_seq_length,
                              current_byte_offset, line_length_excluding_ws,
                              line_length_including_ws))
            long_name = ln_stripped[1:]
            current_short_name = long_name.split()[0]
            current_byte_offset = running_byte_offset
            running_seq_length = 0
        else:
            line_length_including_ws = max(line_length_including_ws, len(ln))
            line_length_excluding_ws = max(line_length_excluding_ws, len(ln_stripped))
            running_seq_length += len(ln_stripped)
    if current_short_name is not None:
        index.append((current_short_name, running_seq_length,
                      current_byte_offset, line_length_excluding_ws,
                      line_length_including_ws))
    return index

Here we use it to index a small multi-FASTA file. We print out the index at the end.

In [ ]:
 fasta_example = StringIO(
'''>sequence1_short_name with optional additional info after whitespace
ACATCACCCCATAAACAAATAGGTTTGGTCCTAGCCTTTCTATTAGCTCTTAGTAAGATTACACATGCAA
GCATCCCCGTTCCAGTGAGTTCACCCTCTAAATCACCACGATCAdAAAGGAACAAGCATCAAGCACGCAGC
AATGCAGCTCAAAACGCTTAGCCTAGCCACACCCCCACGGGAAACAGCAGTGAT
>sequence2_short_name with optional additional info after whitespace
GCCCCAAACCCACTCCACCTTACTACCAGACAACCTTAGCCAAACCATTTACCCAAATAAAGTATAGGCG
ATAGAAATTGAAACCTGGCGCAATAGATATAGTACCGCAAGGGAAAGATGAAAAATTATAACCAAGCATA
ATATAG''')
index_fasta(fasta_example)

What do the fields in those two records mean? Take the first record: ('sequence1_short_name', 194, 69, 70, 71). The fields from left to right are (1) the short name, (2) the length (in nucleotides), (3) the byte offset in the FASTA file of the first nucleotide of the sequence, (4) the maximum number of nucleotides per line, and (5) the maximum number of bytes per line, including whitespace. It's not hard to convince yourself that, if you know all these things, it's not hard to figure out the byte offset of any position in any of the sequences. (This is what the get member of the FastaIndexed class defined below does.)

A typical way to build a FASTA index like this is to use SAMtools, specifically the samtools faidx command. This and all the other samtools commands are documented in its manual.

When you use a tool like this to index a FASTA file, a new file containing the index is written with an additional .fai extension. E.g. if the FASTA file is named hg19.fa, then running samtools faidx hg19.fa will create a new file hg19.fa.fai containing the index.

## Parsing FASTA using PYSAM python module

In [ ]:
import pysam

In [ ]:
# Pysam Fastafile parser
# Random access to fasta formatted files that have been previously indexed
fasta = pysam.Fastafile("Ensembl.GRCh37.fasta") 

In [ ]:
# List with the names of reference sequences (contigs)
fasta.references

In [ ]:
# Number of reference sequences in the file
fasta.nreferences

In [ ]:
# List with the lengths of reference sequences
fasta.lengths

In [ ]:
# Fetch sequences in a region
#fasta['1'][248755121]
fasta.fetch('1', 7, 17)


**Link**: http://pysam.readthedocs.io/en/latest/api.html#pysam.FastaFile

### Basic format
Here's a single sequencing read in FASTQ format:

    @ERR294379.100739024 HS24_09441:8:2203:17450:94030#42/1
    AGGGAGTCCACAGCACAGTCCAGACTCCCACCAGTTCTGACGAAATGATGAGAGCTCAGAAGTAACAGTTGCTTTCAGTCCCATAAAAACAGTCCTACAA
    +
    BDDEEF?FGFFFHGFFHHGHGGHCH@GHHHGFAHEGFEHGEFGHCCGGGFEGFGFFDFFHBGDGFHGEFGHFGHGFGFFFEHGGFGGDGHGFEEHFFHGE

It's spread across four lines.  The four lines are:

1. "`@`" followed by a read name
2. Nucleotide sequence
3. "`+`", possibly followed by some info, but ignored by virtually all tools
4. Quality sequence (explained below)

Here is a very simple Python function for parsing file of FASTQ records:

In [ ]:
def parse_fastq(f):
    reads = []
    while True:
        first_line = f.readline()
        if len(first_line) == 0:
            break
        name = first_line[1:].rstrip()
        seq = f.readline().rstrip()
        # skip + line
        f.readline()
        qual = f.readline().rstrip()
        reads.append((name, seq, qual))
        
    return reads
        

In [ ]:
fastq_string = '''@ERR294379.100739024 HS24_09441:8:2203:17450:94030#42/1
AGGGAGTCCACAGCACAGTCCAGACTCCCACCAGTTCTGACGAAATGATG
+
BDDEEF?FGFFFHGFFHHGHGGHCH@GHHHGFAHEGFEHGEFGHCCGGGF
@ERR294379.136275489 HS24_09441:8:2311:1917:99340#42/1
CTTAAGTATTTTGAAAGTTAACATAAGTTATTCTCAGAGAGACTGCTTTT
+
@@AHFF?EEDEAF?FEEGEFD?GGFEFGECGE?9H?EEABFAG9@CDGGF
@ERR294379.97291341 HS24_09441:8:2201:10397:52549#42/1
GGCTGCCATCAGTGAGCAAGTAAGAATTTGCAGAAATTTATTAGCACACT
+
CDAF<FFDEHEFDDFEEFDGDFCHD=GHG<GEDHDGJFHEFFGEFEE@GH'''

from io import StringIO

parse_fastq(StringIO(fastq_string))


The nucleotide string can sometimes contain the character "N".  N essentially means "no confidence." The sequencer knows there's a nucleotide there but doesn't know whether it's an A, C, G or T.

**Read name** <br/>
Read names often contain information about:

The scientific study for which the read was sequenced. E.g. the string ERR294379 (an SRA accession number) in the read names correspond to this study.
The sequencing instrument, and the exact part of the sequencing instrument, where the DNA was sequenced. See the FASTQ format wikipedia article for specifics on how the Illumina software encodes this information.
Whether the read is part of a paired-end read and, if so, which end it is. Paired-end reads will be discussed further below. The /1 you see at the end of the read names above indicate the read is the first end from a paired-end read.


**Quality values** <br/>
Quality values are probabilities. Each nucleotide in each sequencing read has an associated quality value. A nucleotide's quality value encodes the probability that the nucleotide was incorrectly called by the sequencing instrument and its software. If the nucleotide is A, the corresponding quality value encodes the probability that the nucleotide at that position is actually not an A.

Quality values encoded in two senses: first, the relevant probabilities are rescaled using the Phread scale, which is a negative log scale. In other words if p us the probability that the nucleotide was incorrectly called, we encode this as Q where Q = -10 * log10(p).

For example, if Q = 30, then p = 0.001, a 1-in-1000 chance that the nucleotide is wrong. If Q = 20, then p = 0.01, a 1-in-100 chance. If Q = 10, then p = 0.1, a 1-in-10 chance. And so on.

Second, scaled quality values are rounded to the nearest integer and encoded using ASCII printable characters. For example, using the Phred33 encoding (which is by far the most common), a Q of 30 is encoded as the ASCII character with code 33 + 30 = 63, which is "?". A Q of 20 is encoded as the ASCII character with code 33 + 20 = 53, which is "5". And so on.

## Parsing FASTQ using PYSAM python module


In [ ]:
# Stream access to FASTQ formatted files
fastq = pysam.FastxFile("example_human_Illumina.pe_1.fastq")

In [ ]:
for entry in fastq:
    print(entry.name)
    print(entry.sequence)
    print(entry.comment)
    print(entry.quality)
    break

**Link**: http://pysam.readthedocs.io/en/latest/api.html#fastq-files

## ADDITIONAL

In [ ]:
import re

class FastaOOB(Exception):
    """ Out-of-bounds exception for FASTA sequences """
    
    def __init__(self, value):
        self.value = value
    
    def __str__(self):
        return repr(self.value)

class FastaIndexed(object):
    """ Encapsulates a set of indexed FASTA files.  Does not load the FASTA
        files into memory but still allows the user to extract arbitrary
        substrings, with the help of the index. """
    
    __removeWs = re.compile(r'\s+')
    
    def __init__(self, fafns):
        self.fafhs = {}
        self.faidxs = {}
        self.chr2fh = {}
        self.offset = {}
        self.lens = {}
        self.charsPerLine = {}
        self.bytesPerLine = {}
        
        for fafn in fafns:
            # Open FASTA file
            self.fafhs[fafn] = fh = open(fafn, 'r')
            # Parse corresponding .fai file
            with open(fafn + '.fai') as idxfh:
                for ln in idxfh:
                    toks = ln.rstrip().split()
                    if len(toks) == 0:
                        continue
                    assert len(toks) == 5
                    # Parse and save the index line
                    chr, ln, offset, charsPerLine, bytesPerLine = toks
                    self.chr2fh[chr] = fh
                    self.offset[chr] = int(offset) # 0-based
                    self.lens[chr] = int(ln)
                    self.charsPerLine[chr] = int(charsPerLine)
                    self.bytesPerLine[chr] = int(bytesPerLine)
    
    def __enter__(self):
        return self
    
    def __exit__(self, type, value, traceback):
        # Close all the open FASTA files
        for fafh in self.fafhs.values():
            fafh.close()
    
    def has_name(self, refid):
        return refid in self.offset
    
    def name_iter(self):
        return self.offset.keys()
    
    def length_of_ref(self, refid):
        return self.lens[refid]
    
    def get(self, refid, start, ln):
        ''' Return the specified substring of the reference. '''
        assert refid in self.offset
        if start + ln > self.lens[refid]:
            raise ReferenceOOB('"%s" has length %d; tried to get [%d, %d)' % (refid, self.lens[refid], start, start + ln))
        fh, offset, charsPerLine, bytesPerLine = \
            self.chr2fh[refid], self.offset[refid], \
            self.charsPerLine[refid], self.bytesPerLine[refid]
        byteOff = offset
        byteOff += (start // charsPerLine) * bytesPerLine
        into = start % charsPerLine
        byteOff += into
        fh.seek(byteOff)
        left = charsPerLine - into
        # Count the number of line breaks interrupting the rest of the
        # string we're trying to read
        if ln < left:
            return fh.read(ln)
        else:
            nbreaks = 1 + (ln - left) // charsPerLine
            res = fh.read(ln + nbreaks * (bytesPerLine - charsPerLine))
            res = re.sub(self.__removeWs, '', res)
        return res

In [ ]:
# first we'll write a new FASTA file
with open('tmp.fa', 'w') as fh:
    fh.write('''>sequence1_short_name with optional additional info after whitespace
ACATCACCCCATAAACAAATAGGTTTGGTCCTAGCCTTTCTATTAGCTCTTAGTAAGATTACACATGCAA
GCATCCCCGTTCCAGTGAGTTCACCCTCTAAATCACCACGATCAAAAGGAACAAGCATCAAGCACGCAGC
AATGCAGCTCAAAACGCTTAGCCTAGCCACACCCCCACGGGAAACAGCAGTGAT
>sequence2_short_name with optional additional info after whitespace
GCCCCAAACCCACTCCACCTTACTACCAGACAACCTTAGCCAAACCATTTACCCAAATAAAGTATAGGCG
ATAGAAATTGAAACCTGGCGCAATAGATATAGTACCGCAAGGGAAAGATGAAAAATTATAACCAAGCATA
ATATAG''')
with open('tmp.fa') as fh:
    idx = index_fasta(fh)
with open('tmp.fa.fai', 'w') as fh:
    fh.write('\n'.join(['\t'.join(map(str, x)) for x in idx]))
with FastaIndexed(['tmp.fa']) as fa_idx:
    print (fa_idx.get('sequence2_short_name', 100, 30))